In [1]:
import json
from collections import defaultdict

In [2]:
with open("countries.geojson", "r") as f:
    data = json.load(f)

In [3]:
def get_points(geom):
    #geom = d['geometry']
    coords = None
    # Convert polygons to multipolygons, to make our life easier...
    if geom['type'] == "Polygon":
        polys = [geom['coordinates']]
    else:
        polys = geom['coordinates']
        
    points = {
        (p[0], p[1]) 
        for poly in polys
        for something in poly
        for p in something
    }
    return points

In [4]:
countries_coords = [{"id": c['properties']['SU_A3'],
                     "polygon_coords": get_points(c['geometry'])
                    }
                    for c in data["features"]]

In [5]:
# countries_test = [d for d in countries_coords if d['id'] in ['FXX', 'ESP']]
# countries_test

In [6]:
coord_country_map = defaultdict(lambda: [])

# Try just a dumb thing of checking coordinates that occur in other countries' boundaries.
for country in countries_coords:
    c_id = country['id']
    for point in set(country['polygon_coords']):
        coord_country_map[point].append(c_id)

In [7]:
edges = defaultdict(lambda: set())
for k, v in coord_country_map.items():
    # Go through all pairs of country ids in these points
    for i in range(len(v)-1):
        for j in range(i+1, len(v)):
            edges[v[i]].add(v[j])
            edges[v[j]].add(v[i])

In [8]:
edges

defaultdict(<function __main__.<lambda>()>,
            {'ZWE': {'BWA', 'MOZ', 'NAM', 'ZAF', 'ZMB'},
             'ZMB': {'AGO', 'BWA', 'COD', 'MOZ', 'MWI', 'NAM', 'TZA', 'ZWE'},
             'ZAF': {'BWA', 'LSO', 'MOZ', 'NAM', 'SWZ', 'ZWE'},
             'BWA': {'NAM', 'ZAF', 'ZMB', 'ZWE'},
             'MOZ': {'MWI', 'SWZ', 'TZA', 'ZAF', 'ZMB', 'ZWE'},
             'NAM': {'AGO', 'BWA', 'ZAF', 'ZMB', 'ZWE'},
             'COD': {'AGO',
              'BDI',
              'CAF',
              'COG',
              'RWA',
              'SDS',
              'TZA',
              'UGA',
              'ZMB'},
             'AGO': {'COD', 'COG', 'NAM', 'ZMB'},
             'TZA': {'BDI', 'COD', 'KEN', 'MOZ', 'MWI', 'RWA', 'UGA', 'ZMB'},
             'MWI': {'MOZ', 'TZA', 'ZMB'},
             'YEM': {'OMN', 'SAU'},
             'SAU': {'ARE', 'IRQ', 'JOR', 'KWT', 'OMN', 'QAT', 'YEM'},
             'OMN': {'ARE', 'SAU', 'YEM'},
             'VNM': {'CHN', 'KHM', 'LAO'},
             'KHM': {'LAO

In [9]:
# Convert sets to lists.
edges_write = {
    k: list(vals) for k, vals in edges.items()
}

# write to file!
with open("country_adjacency.json", "w") as f:
    f.write(json.dumps(edges_write))